In [1]:
import pandas

In [2]:
df = pandas.read_csv('cards.csv')


/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,8,14,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df = df.drop(columns='index')

In [6]:
df.head()

,id,artist,borderColor,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,...,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
0,1,Rebecca Guay,black,G,NaN,G,4.0,NaN,983.0,NaN,...,NaN,NaN,15023.0,"If you would draw a card, you may instead choo...",NaN,Enchantment,Enchantment,1669af17-d287-5094-b005-4b143441442f,NaN,NaN
1,2,Stephen Daniele,black,U,NaN,U,3.0,NaN,10482.0,NaN,...,"Human,Wizard",NaN,15024.0,When Academy Researchers enters the battlefiel...,2,Creature — Human Wizard,Creature,047d5499-a21c-5f5c-9679-1599fcaf9815,NaN,NaN
2,3,John Avon,black,"U,W",NaN,NaN,0.0,NaN,568.0,NaN,...,NaN,NaN,15025.0,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...,NaN,Land,Land,ee19938c-4007-58f1-8904-fae28007b422,NaN,NaN
3,4,Roger Raupp,black,B,NaN,B,3.0,NaN,15710.0,NaN,...,NaN,NaN,15026.0,Target creature gets -1/-1 until end of turn.\...,NaN,Instant,Instant,8774e18f-3752-5c06-af94-5da3960da9ed,NaN,NaN
4,5,Christopher Moeller,black,G,NaN,G,2.0,NaN,9841.0,NaN,...,NaN,NaN,15027.0,Target creature gets +1/+1 until end of turn.\...,NaN,Instant,Instant,4e875bca-0c52-5d60-889d-1db67e261737,NaN,NaN


In [9]:
df.columns

Index(['id', 'artist', 'borderColor', 'colorIdentity', 'colorIndicator',
       'colors', 'convertedManaCost', 'duelDeck', 'edhrecRank',
       'faceConvertedManaCost', 'flavorText', 'frameEffect', 'frameEffects',
       'frameVersion', 'hand', 'hasFoil', 'hasNoDeckLimit', 'hasNonFoil',
       'isAlternative', 'isArena', 'isBuyABox', 'isDateStamped', 'isFullArt',
       'isMtgo', 'isOnlineOnly', 'isOversized', 'isPaper', 'isPromo',
       'isReprint', 'isReserved', 'isStarter', 'isStorySpotlight',
       'isTextless', 'isTimeshifted', 'layout', 'leadershipSkills', 'life',
       'loyalty', 'manaCost', 'mcmId', 'mcmMetaId', 'mtgArenaId', 'mtgoFoilId',
       'mtgoId', 'multiverseId', 'name', 'names', 'number', 'originalText',
       'originalType', 'otherFaceIds', 'power', 'printings', 'purchaseUrls',
       'rarity', 'scryfallId', 'scryfallIllustrationId', 'scryfallOracleId',
       'setCode', 'side', 'subtypes', 'supertypes', 'tcgplayerProductId',
       'text', 'toughness', 'type', '

In [11]:
df.flavorText.dropna()

1        They brandish their latest theories as warrior...
3        One rarely notices a heartbeat, save when it i...
4        The power of the wild, concentrated in a singl...
5        In the aftermath of war, when the slaying is l...
6        "The East Wind, an interloper in the dominions...
                               ...                        
50405    "The sky hedrons are covered with tern nests. ...
50407    The merfolk call the sky goddess Emeria. The k...
50408    "The best spot to hook one is right behind the...
50409    "Why fight the world when you know who will wi...
50411    Expeditions follow the paths it leaves in its ...
Name: flavorText, Length: 28026, dtype: object